In [ ]:
%matplotlib inline

import h2o
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.automl import H2OAutoML

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# constant random seed 
seed = 3232

# Connect to a pre-existing cluster
h2o.init()

In [ ]:
from h2o.utils.shared_utils import _locate # private function. used to find files within h2o git project directory.

#df = h2o.import_file(path=_locate("storymanagement.csv"))
df = h2o.import_file(path=_locate("infra_old.csv"))
# df = dfi.na_omit()


In [ ]:
df.describe()

In [25]:
# Does preprocessing on the raw data frame
def preprocess(df):
    df_edited = df
    
    for field in ["Id", "Current State", "Requested By", "URL", "Zendesk ID", "Integration", "Labels",
                  "Owned By", "Owned By 2", "Owned By 3", "Iteration Start", "Iteration End", "Created at", 
                  "Accepted at", "Deadline", "Title", "Description"
                 ]: 
        try:
            df_edited = df_edited.drop(field)
        except:
            pass

    # Convert fields to logical factors
    df_edited['Type'] = df_edited['Type'].asfactor() #drop non-features altogether?
   # df_edited['Estimate'] = df_edited['Estimate'].asfactor() #drop non-features altogether?
    
    return df_edited


In [26]:
df_factors = preprocess(df)

In [27]:
# Set Stop Words
# The STOP WORDS we are importing are from the nltk package
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

# Break titles into sequence of words
descriptions = tokenize(df["Description"].ascharacter())
titles = tokenize(df["Title"].ascharacter())

x = df["Description"].ascharacter().cbind(df["Title"].ascharacter())
all_text = tokenize(x)
titles.head()
descriptions.head()
all_text.head()

C1
get
prioritization
working
""
color
code
iterations
grey
past
blue


In [51]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

# This takes time to run - left commented out
description_w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
description_w2v_model.train(training_frame=descriptions)

# title_w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
# title_w2v_model.train(training_frame=titles)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [114]:
description_w2v_model.find_synonyms("jasmine", count = 5)

OrderedDict()

In [53]:
# title_w2v_model.find_synonyms("count", count=5)

In [54]:
description_vecs = description_w2v_model.transform(descriptions, aggregate_method = "AVERAGE")
title_vecs = description_w2v_model.transform(titles, aggregate_method = "AVERAGE")
title_vecs.names = ["title_" + t for t in title_vecs.names]

title_vecs

title_C1,title_C2,title_C3,title_C4,title_C5,title_C6,title_C7,title_C8,title_C9,title_C10,title_C11,title_C12,title_C13,title_C14,title_C15,title_C16,title_C17,title_C18,title_C19,title_C20,title_C21,title_C22,title_C23,title_C24,title_C25,title_C26,title_C27,title_C28,title_C29,title_C30,title_C31,title_C32,title_C33,title_C34,title_C35,title_C36,title_C37,title_C38,title_C39,title_C40,title_C41,title_C42,title_C43,title_C44,title_C45,title_C46,title_C47,title_C48,title_C49,title_C50,title_C51,title_C52,title_C53,title_C54,title_C55,title_C56,title_C57,title_C58,title_C59,title_C60,title_C61,title_C62,title_C63,title_C64,title_C65,title_C66,title_C67,title_C68,title_C69,title_C70,title_C71,title_C72,title_C73,title_C74,title_C75,title_C76,title_C77,title_C78,title_C79,title_C80,title_C81,title_C82,title_C83,title_C84,title_C85,title_C86,title_C87,title_C88,title_C89,title_C90,title_C91,title_C92,title_C93,title_C94,title_C95,title_C96,title_C97,title_C98,title_C99,title_C100
0.00850448,-0.376571,0.240607,0.0594223,-0.0320608,-0.0591226,0.098953,0.055899,-0.0624478,0.0661417,-0.278999,0.363803,-0.0363973,0.107152,0.0715517,0.243875,-0.0802126,-0.284127,-0.104366,-0.0141464,0.205823,-0.282788,0.0819282,-0.0513203,0.186569,-0.10225,-0.243165,0.25329,0.0196712,-0.125477,0.185177,0.00278565,0.0208074,0.141285,0.215161,-0.20114,0.166351,0.0694677,0.140167,-0.20838,-0.202521,0.280839,0.150747,0.179097,-0.0812454,-0.108472,0.0278634,-0.114406,0.092723,0.0881863,0.143446,-0.145181,-0.127485,0.232606,-0.0302365,0.0758815,0.120763,0.0513126,-0.196285,0.0518081,0.0391249,0.193377,0.0116519,-0.0388775,0.00973701,0.0615587,0.0793442,-0.137483,0.13759,0.231341,-0.0898565,0.419892,-0.254763,-0.242648,0.141035,0.0762604,-0.124371,0.136005,-0.072098,-0.0396879,0.167848,0.0548347,0.148509,0.0432684,0.0308584,0.350314,-0.264049,-0.319718,-0.309317,-0.354438,-0.2804,0.153175,0.0608864,-0.0918857,-0.14531,0.0164998,-0.0524767,0.075167,0.168136,0.0379977
0.0765456,-0.027072,0.0444293,-0.255209,0.169295,0.0664769,-0.0292822,0.267601,-0.088926,-0.317899,-0.105346,0.00159404,0.0769052,0.0551184,0.0673638,0.0261637,-0.145594,-0.0222049,-0.323634,0.0846212,-0.105241,-0.00120874,0.0398806,0.0506924,0.205435,-0.133132,-0.095914,-0.0374247,0.228699,-0.0574891,0.121848,-0.0708098,0.15807,-0.0264842,0.137727,0.192793,0.229943,-0.0634152,0.107787,-0.0843162,-0.128605,-0.141498,0.351497,0.00288611,0.165196,0.0381958,-0.152677,0.0432304,0.269283,0.0888247,0.0822012,-0.166064,0.0294299,0.13539,-0.0138749,0.0844726,0.00535169,0.286056,-0.0308375,-0.08461,-0.117077,0.111472,-0.064649,-0.0198523,0.00612759,-0.145911,-0.0980286,-0.0110718,0.195518,0.0606558,0.169958,-0.0222366,0.0722312,-0.336685,-0.00328592,0.0358739,0.0040909,0.0234808,0.174689,0.0234279,0.257153,-0.0952196,-0.00724689,-0.0305632,-0.057611,-0.000635722,-0.173781,-0.247999,-0.222845,-0.354478,0.00354825,0.215468,0.0987957,0.0283362,0.0113556,0.0810819,0.139532,-0.00356843,0.405739,0.248826
0.223165,-0.126702,-0.0430353,0.215402,0.170595,0.127507,0.100199,0.239805,-0.0352299,-0.0380167,-0.0476617,0.000893425,0.120723,-0.102155,0.126136,0.52258,0.109111,0.0260142,-0.0401885,-0.0307215,0.313006,-0.192509,-0.0140025,0.210266,0.126307,-0.403165,-0.0892678,-0.0750582,0.0505759,0.0786226,0.249358,-0.0540406,0.180655,-0.00416084,0.0468938,0.161711,0.217425,-0.206062,-0.0276089,-0.0624749,-0.227305,0.0244669,0.134052,-0.0215993,-0.155285,0.00296798,0.0834216,0.133989,0.034025,-0.136898,0.115679,0.120897,-0.0474875,0.108134,-0.0706961,-0.0865675,0.132854,0.0319247,-0.252757,0.0157442,-0.157993,0.00117513,-0.113746,-0.205542,0.0477652,-0.0800849,0.00511651,0.0742756,0.018431,-0.0810128,0.264214,-0.194647,-0.228266,-0.220715,-0.038519,0.225087,-0.0693953,0.252984,0.0972238,-0.0471392,0.204746,-0.229812,0.0657142,-0.0098466,0.179803,0.0685476,-0.160126,-0.244607,-0.328315,-0.142647,-0.0273058,-0.0310751,0.117097,0.0431557,-0.11698,-0.136901,-0.164908,0.25254,0.213831,-0.0344707
0.205089,-0.193578,0.0848697,-0.

In [55]:
df_edited = df_factors.cbind(description_vecs)
df_edited_title = df_factors.cbind(description_vecs).cbind(title_vecs)

df_edited.head()

Iteration,Type,Estimate,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,bug,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,feature,2,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,

In [56]:

train,test,valid = df_edited.split_frame(ratios=[.85, .1], seed = seed)
train_title,test_title,valid_title = df_edited_title.split_frame(ratios=[.85, .1], seed = seed)

In [57]:
train_title.na_omit()
# See that the data is ready
train_title.describe()

Rows:11939
Cols:203




,Iteration,Type,Estimate,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100,title_C1,title_C2,title_C3,title_C4,title_C5,title_C6,title_C7,title_C8,title_C9,title_C10,title_C11,title_C12,title_C13,title_C14,title_C15,title_C16,title_C17,title_C18,title_C19,title_C20,title_C21,title_C22,title_C23,title_C24,title_C25,title_C26,title_C27,title_C28,title_C29,title_C30,title_C31,title_C32,title_C33,title_C34,title_C35,title_C36,title_C37,title_C38,title_C39,title_C40,title_C41,title_C42,title_C43,title_C44,title_C45,title_C46,title_C47,title_C48,title_C49,title_C50,title_C51,title_C52,title_C53,title_C54,title_C55,title_C56,title_C57,title_C58,title_C59,title_C60,title_C61,title_C62,title_C63,title_C64,title_C65,title_C66,title_C67,title_C68,title_C69,title_C70,title_C71,title_C72,title_C73,title_C74,title_C75,title_C76,title_C77,title_C78,title_C79,title_C80,title_C81,title_C82,title_C83,title_C84,title_C85,title_C86,title_C87,title_C88,title_C89,title_C90,title_C91,title_C92,title_C93,title_C94,title_C95,title_C96,title_C97,title_C98,title_C99,title_C100
type,int,enum,int,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real,real
mins,1.0,,0.0,-1.06561160088,-0.620505332947,-0.477536082268,-1.14697313309,-0.701708972454,-0.694024324417,-0.492218822241,-0.821995913982,-0.806790530682,-0.636340975761,-0.784666001797,-0.648594975471,-0.605274498463,-0.743754029274,-0.459112197161,-0.566463470459,-0.702475607395,-0.794710218906,-0.629830658436,-0.646800637245,-0.684366643429,-0.546742081642,-0.489897787571,-0.709513366222,-0.569203853607,-0.648339807987,-0.770271003246,-0.576652824879,-0.497763842344,-0.533407688141,-0.411487191916,-0.591594994068,-0.33145236969,-0.6005615592,-0.496365696192,-0.712744832039,-0.723688483238,-0.7658624053,-0.85724645853,-0.724238097668,-0.761876940727,-0.457559406757,-0.311737537384,-0.656483829021,-0.484415948391,-0.388619184494,-0.47151646018,-0.510570347309,-0.524843871593,-0.737192332745,-0.500103235245,-0.455345213413,-0.720468997955,-0.569995999336,-0.722967684269,-0.457958608866,-0.580211818218,-0.685309708118,-0.575700342655,-1.16587615013,-0.7707015872,-0.608982145786,-0.479100614786,-0.688865542412,-0.511053442955,-0.582102239132,-0.583456635475,-0.592621445656,-0.299879163504,-0.385085344315,-0.640226244926,-0.478908896446,-0.405673712492,-0.658492207527,-0.550942718983,-0.53025829792,-0.598512530327,-0.360642194748,-0.810838580132,-0.46425974369,-0.422770023346,-0.620904743671,-0.640080392361,-0.611474692822,-0.528234541416,-0.480566740036,-0.545552253723,-0.87810665369,-0.560658097267,-0.731715381145,-0.715015172958,-0.883855938911,-0.685586929321,-0.566116690636,-0.767908453941,-0.68895149231,-0.517461419106,-0.590309500694,-0.4370

In [58]:
predictors = ["Iteration", "Type"] + description_vecs.names
predictors_title = ["Iteration", "Type"] + title_vecs.names

response = "Estimate"

# gbm_embeddings = H2OGradientBoostingEstimator(stopping_metric = "AUC", stopping_tolerance = 0.001,
#                                               stopping_rounds = 5, score_tree_interval = 10,
#                                               model_id = "gbm_embeddings.hex"
#                                              )

In [59]:
# # Run GBM
# gbm_model = H2OGradientBoostingEstimator(distribution = "gaussian", ntrees=100, learn_rate=.01)
# gbm_model.train(x=predictors, y="Estimate", training_frame=train, validation_frame=valid)

# # Run GBM for title
# gbm_model_title = H2OGradientBoostingEstimator(distribution = "gaussian", ntrees=100, learn_rate=.01)
# gbm_model_title.train(x=predictors_title, y="Estimate", training_frame=train_title, validation_frame=valid_title)


In [60]:
# # print("With Embeddings AUC: " + str(round(gbm_model.auc(valid = True), 3)))
# gbm_metrics = gbm_model.model_performance(test)
# gbm_metrics

In [61]:
# gbm_metrics_title = gbm_model_title.model_performance(test_title)
# gbm_metrics_title

In [68]:
# Run AutoML
x = train.columns
y = "Estimate"
x.remove(y)
aml = H2OAutoML(max_runtime_secs = 30, seed = seed)
aml.train(x = x, y = y,
          training_frame = train,
          validation_frame = valid,
          leaderboard_frame = test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [69]:
# # Run AutoML
# x = train_title.columns
# y = "Estimate"
# x.remove(y)
# aml_title = H2OAutoML(max_runtime_secs = 30, seed = seed)
# aml_title.train(x = x, y = y,
#           training_frame = train_title,
#           validation_frame = valid_title,
#           leaderboard_frame = test_title)

In [70]:

lb = aml.leaderboard
aml_leader_metrics = aml.leader.model_performance(test)

aml_leader_metrics


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 1.04753628491
RMSE: 1.02349220071
MAE: 0.70139783744
RMSLE: 0.455294469971
R^2: 0.0173571701673
Mean Residual Deviance: 1.04753628491
Null degrees of freedom: 472
Residual degrees of freedom: 470
Null deviance: 504.623837566
Residual deviance: 495.484662762
AIC: 1372.28245097


In [71]:
# lb = aml_title.leaderboard
# aml_title_leader_metrics = aml_title.leader.model_performance(test_title)

# aml_title_leader_metrics

In [79]:
def predict(stories, w2v):
    
    words = tokenize(stories["Description"].ascharacter())
    description_vec = w2v.transform(words, aggregate_method="AVERAGE")
    
#     summary_words = tokenize(reviews["Summary"].ascharacter())
#     summary_vec = w2v.transform(summary_words, aggregate_method="AVERAGE")
    
    model_data = stories.cbind(description_vec)#.cbind(summary_vec)
    print(aml.leader.predict(model_data))

#Iteration, type, description
stories = h2o.H2OFrame([
    [15, "feature", "Allow account owners and admins to view and delete github integrations. Super admins can view but not delete", "This includes when a user is looking at the list of github integrations, clicking on a github integration should take the user to the edit or view page depending."],
    [289, "feature", "Allow account owners and admins to view and delete github integrations. Super admins can view but not delete", "This includes when a user is looking at the list of github integrations, clicking on a github integration should take the user to the edit or view page depending."],
    [15, "feature", "Store education box preference via App Settings", "Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],
    [289, "feature", "Store education box preference via App Settings","Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],

])

stories2 = h2o.H2OFrame([
    [15, "feature", "Create the omega primary and warehouse Cloud SQL databases via terraform","Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],
    [647, "feature", "Create the omega primary and warehouse Cloud SQL databases via terraform","Now that we've seen it work locally, let's add the appropriate logic to the AppSettings endpoint and persist it.\nDo not show preference:\n\nLast time\nRemind me again"],
    [647, "feature", "As a tracker user I can add an emoji as an attachment","Look into how it was done for google attachments. This will involve creating new command and bumping the version of the command layer. User experience must be perfect. github.com/someexample. Acceptance criteria: open the attachment menu. select emoji. type every single emoji possible. save the story or epic. see that it is attached"],
    [1, "feature", "As a Tracker Team Member, I can view a Concourse instance managed by our Infra Team","Spin up in the cf-tracker-production-services project with n >= 1 workers.\n\nAuthentication beyond concourse admin user is out of scope.\n\n## Acceptance criteria\nA concourse instance spun up in the `cf-tracker-production-services` project is accessible via the browser.\n\n---Docs added as part of this story:* https://github.com/pivotaltracker/tracker-docs/blob/master/devops/HowTos/credhub-usage.md* https://github.com/pivotaltracker/tracker-docs/blob/master/devops/HowTos/cloud-workstation-login.md* https://github.com/pivotaltracker/tracker/blob/master/nonapp/concourse-deployment/README.md"],
    [700, "feature", "As a Tracker Team Member, I can view a Concourse instance managed by our Infra Team","Spin up in the cf-tracker-production-services project with n >= 1 workers.\n\nAuthentication beyond concourse admin user is out of scope.\n\n## Acceptance criteria\nA concourse instance spun up in the `cf-tracker-production-services` project is accessible via the browser.\n\n---Docs added as part of this story:* https://github.com/pivotaltracker/tracker-docs/blob/master/devops/HowTos/credhub-usage.md* https://github.com/pivotaltracker/tracker-docs/blob/master/devops/HowTos/cloud-workstation-login.md* https://github.com/pivotaltracker/tracker/blob/master/nonapp/concourse-deployment/README.md"],
    [400, "feature", "Add a new story state","Add a new story state to represent the testing state. It should appear in the drop down after delivered but as a separate step than accept/reject. Stories with this state should display in order before stories in accept/reject but afte stories that are started and delivered. Let's move them automatically, and animate that move so that users know what is going on. Should work on all browsers - chrome/firefox/ie6 etc. Acceptance criteria: Set a story to  the testing state. pass/fail. see that the state is refelcted on the story and everything else works great."],

])

stories.col_names=["Iteration", "Type", "Title", "Description"]
stories2.col_names=["Iteration", "Type", "Title", "Description"]

predict(stories,description_w2v_model)
predict(stories2,description_w2v_model)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
1.09844
0.912773
1.12267
1.14035



stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
1.12267
0.742474
1.07922
0.980533
0.954956
1.40263


In [80]:
 aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_0_AutoML_20181017_092238,1.04754,1.02349,1.04754,0.701398,0.455294
StackedEnsemble_AllModels_0_AutoML_20181017_092238,1.04754,1.02349,1.04754,0.701398,0.455294
StackedEnsemble_AllModels_0_AutoML_20181017_092436,1.04779,1.02362,1.04779,0.701548,0.455296
DRF_0_AutoML_20181017_092238,1.09096,1.04449,1.09096,0.72673,0.463249
DRF_0_AutoML_20181017_092436,1.09096,1.04449,1.09096,0.72673,0.463249
XRT_0_AutoML_20181017_092238,1.10086,1.04922,1.10086,0.734296,0.464002
XRT_0_AutoML_20181017_092436,1.10086,1.04922,1.10086,0.734296,0.464002
